In [1]:
import jieba
from snownlp import SnowNLP # 使用
from snownlp import seg  # 分词库
from snownlp import sentiment # 情感分词
from snownlp import normal #停用词处理

In [2]:
import tushare as ts
ts.set_token('0c1528160ad2b51ea43872ded226137ff7ece65d54d806db191d68e6')
pro=ts.pro_api()
data = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
# data
stock_list = list(data['ts_code'])[:1000]
stock_str = ",".join(stock_list)

In [3]:
change = pro.daily(ts_code='000001.SZ',start_date='20201225',end_date='20211225')

In [43]:
# change

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20211224,17.33,17.35,17.21,17.31,17.32,-0.01,-0.0577,488251.49,843109.664
1,000001.SZ,20211223,17.40,17.43,17.21,17.32,17.39,-0.07,-0.4025,1059575.94,1831122.716
2,000001.SZ,20211222,17.62,17.64,17.30,17.39,17.59,-0.20,-1.1370,976927.75,1701772.583
3,000001.SZ,20211221,17.49,17.71,17.48,17.59,17.52,0.07,0.3995,893734.04,1572895.581
4,000001.SZ,20211220,17.56,17.65,17.45,17.52,17.57,-0.05,-0.2846,746435.20,1309029.268
...,...,...,...,...,...,...,...,...,...,...,...
238,000001.SZ,20201231,19.21,19.58,19.02,19.34,19.20,0.14,0.7292,924503.43,1781736.285
239,000001.SZ,20201230,19.00,19.20,18.72,19.20,19.17,0.03,0.1565,978497.78,1854082.812
240,000001.SZ,20201229,18.87,19.30,18.70,19.17,18.85,0.32,1.6976,963092.23,1837947.238
241,000001.SZ,20201228,18.02,18.86,17.96,18.85,18.04,0.81,4.4900,1270337.06,2352947.321


In [72]:
neg_list=[]
pos_list = []

In [55]:
a = pro.anns(ts_code = '600255.SH',start_date='20201225',end_date='20211225')


In [56]:
for i in range(change.shape[0]):
    if(change.iloc[i]['change']<0):
        neg_title = a[a["ann_date"]==change.iloc[i]['trade_date']]['title']
        neg_content = a[a["ann_date"]==change.iloc[i]['trade_date']]['content']
        for n in neg_title:
            neg_list.append(n)
        for n in neg_content:
            neg_list.append(n)

In [71]:
# neg_list

In [70]:
for n in neg_list:
    if(n != ''):
        try:
            with open('neg.txt','a') as f:
                f.write(n+'\n')
        except:
            continue

In [84]:
a = pro.anns(ts_code = '000333.SZ',start_date='20201225',end_date='20211225')

In [85]:
for i in range(change.shape[0]):
    if(change.iloc[i]['change']>0):
        pos_title = a[a["ann_date"]==change.iloc[i]['trade_date']]['title']
        pos_content = a[a["ann_date"]==change.iloc[i]['trade_date']]['content']
        for p in pos_title:
            pos_list.append(p)
        for p in pos_content:
            pos_list.append(p)

In [87]:
# pos_list

In [88]:
for p in pos_list:
    if(p != ''):
        try:
            with open('pos.txt','a') as f:
                f.write(p+'\n')
        except:
            continue

In [4]:
class SnowNLP(object):
    @property
    def sentiments(self):
        return sentiment.classify(self.doc)

In [5]:
class Sentiment(object):
    def handle(self, doc):
        words = seg.seg(doc)
        words = normal.filter_stop(words)
        return words
    
    # 从这里发现，训练和分词，采用的都是 self.handle 函数，如果只修改这里，确实也是可以，还不影响他自带的分词
    # 如果修改seg.seg的话，就得从新训练过
    def train(self, neg_docs, pos_docs):
        data = []
        for sent in neg_docs:
            data.append([self.handle(sent), 'neg'])
        for sent in pos_docs:
            data.append([self.handle(sent), 'pos'])
        self.classifier.train(data)

    def classify(self, sent):
        ret, prob = self.classifier.classify(self.handle(sent))
        if ret == 'pos':
            return prob
        return 1-prob

In [98]:
sentiment.train(r'neg.txt',r'pos.txt')

In [105]:
fname = r'sentiment.marshal'
sentiment.save(fname)    # 保存训练结果
# sentiment.load(fname)  #读取已经训练好的数据

In [6]:
import SnowNLP
import pandas as pd
import numpy as np

In [7]:
a = pro.anns()
nas = pd.DataFrame()
sentiment_list = []
ts_code_list = []
for i in range(a.shape[0]):
    s = SnowNLP(a.iloc[i]['title'])
    sentiment_list.append(s.sentiments)
    ts_code_list.append(a.iloc[i]['ts_code'])
nas['ts_code'] = ts_code_list
nas['sentiment'] = sentiment_list



In [8]:
nas_mean = pd.DataFrame(nas.groupby('ts_code').agg(np.mean))
nas_mean = nas_mean.sort_values(by='sentiment',ascending=False)
nas_mean

,sentiment
ts_code,
000717.SZ,1.000000e+00
000408.SZ,9.999879e-01
600167.SH,9.998626e-01
000566.SZ,9.998163e-01
600298.SH,9.997993e-01
...,...
000695.SZ,1.169147e-03
600259.SH,6.587915e-04
600491.SH,4.259535e-04


In [9]:
code = list(nas_mean.index)
sentiment = list(nas_mean['sentiment'])

In [10]:
import pymysql

def load_all_from_mysql(nas_mean,code,sentiment):
    connect = pymysql.Connect(
        host = 'localhost',
        port = 3306,
        user = 'root',
        password = 'Cliu123#',
        db = 'fintech',
        charset = 'utf8'
    )


    cursor = connect.cursor()
    
    sql_delete = "DELETE FROM finweb_recommenddata"
    cursor.execute(sql_delete)
    connect.commit()
    sql_insert = "INSERT INTO finweb_recommenddata(ii,code,sentiment) VALUES (%s,%s,%s)"
    for i in range(nas_mean.shape[0]):
        cursor.execute(sql_insert,(i,code[i][:-3],sentiment[i]))
        connect.commit()
    connect.close()

if __name__ == '__main__':
    load_all_from_mysql(nas_mean,code,sentiment)

In [9]:
# for i in range(nas_mean.shape[0]):
#     print('INSERT INTO finweb_recommenddata(ii,code,sentiment) VALUES(',i,',',"'",code[i][:-3],"',",sentiment[i],');')

In [ ]:
# # Create your views here.
# a = pro.news(src='sina',start_date='2021-11-21',
#             end_date='2021-11-22',ts_code)
# news_list1 = []
# news_list2 = []
# for i in range(3):
#     dic1 = {}
#     # dic1['classify'] = a.ix[i]['classify']
#     dic1['title']=a.iloc[i]['title']
#     dic1['time']=a.iloc[i]['datetime']
#     # dic1['url']=a.ix[i]['url']
#     dic1['content']=a.iloc[i]['content'][0:100]+"....."
#     dic2 = {}
#     # dic2['classify'] = a.ix[i+3]['classify']
#     dic2['title']=a.iloc[i+3]['title']
#     dic2['time']=a.iloc[i+3]['datetime']
#     # dic2['url']=a.ix[i+3]['url']
#     dic2['content']=a.iloc[i+3]['content'][0:100]+"....."
#     news_list1.append(dic1)
#     news_list2.append(dic2)